In [1]:
import numpy as np, pandas as pd, random
from tqdm import tqdm

# --------------------------------------------------------
# 1) carica
# --------------------------------------------------------
df = pd.read_parquet("positions.parquet")

# --------------------------------------------------------
# 2) normalizza feature 0-1
# --------------------------------------------------------
df["S_path"]  = df["path_distance"]   / df["path_distance"].max()
df["S_goal"]  = df["distance_to_goal"] / np.sqrt(2)
df["S_wall"] = df["distance_from_wall"] / df["distance_from_wall"].max()

df["q_degree"] = (df["degree"] - 1) / 3.0          # 1→0  … 4→1
df["S_deg"]    = 1.0 - df["q_degree"]              # 0 buono, 1 cattivo

# --------------------------------------------------------
# 3) pesi
# --------------------------------------------------------
w_path, w_wall, w_goal, w_deg = 0.5, 0.3, 0.15, 0.05

# --------------------------------------------------------
# 4) score (minore è migliore)
# --------------------------------------------------------
df["total_score"] = (
      -w_path * df.S_path
    +  w_wall * df.S_wall      # bonus: più grande → meglio → sottrai con segno meno
    - w_goal * df.S_goal
    +  w_deg  * df.S_deg       # penalità: più grande → peggio → aggiungi
)

# quick sanity
print(df.nsmallest(3,"total_score")[["x","y","degree","total_score"]])
print(df.nlargest (3,"total_score")[["x","y","degree","total_score"]])

# --------------------------------------------------------
# 5) genera 200k coppie (50 % facili / 50 % difficili)
# --------------------------------------------------------
def sample_pairs(df, num_pairs=200_000, hard_ratio=0.7, thresh=0.05):
    idx     = df.index.to_numpy()
    scores  = df["total_score"].to_numpy()
    pairs   = set()
    n_hard  = int(num_pairs * hard_ratio)
    pbar    = tqdm(total=num_pairs, desc="sampling pairs")
    while len(pairs) < num_pairs:
        i, j = np.random.choice(idx, 2, replace=False)
        si, sj = scores[i], scores[j]
        if si == sj:          # pari → ignora
            continue
        better, worse = (i, j) if si < sj else (j, i)  # score minore è migliore
        delta = abs(si - sj)
        want_hard = len(pairs) < n_hard
        if (want_hard and delta < thresh) or (not want_hard and delta >= thresh):
            pair = (df.at[better,"x"], df.at[better,"y"],
                    df.at[worse,"x"],  df.at[worse,"y"], 1)
            if pair not in pairs:
                pairs.add(pair); pbar.update(1)
    pbar.close()
    return list(pairs)

pairs = sample_pairs(df, num_pairs=200_000)

# --------------------------------------------------------
# 6) salva
# --------------------------------------------------------
pd.DataFrame(pairs,
    columns=["x_better","y_better","x_worse","y_worse","preference"]
).to_parquet("preferences.parquet", index=False)

print("✓ preferences.parquet salvato con", len(pairs), "coppie")


             x         y  degree  total_score
3229  0.014095  0.000662       1    -0.588133
827   0.000708  0.013428       1    -0.587952
3575  0.072639  0.001336       1    -0.580469
             x         y  degree  total_score
3380  0.940444  0.947991       2     0.288656
1459  0.941289  0.854145       3     0.281963
5683  0.924581  0.944208       2     0.279894


sampling pairs: 100%|██████████| 200000/200000 [01:08<00:00, 2900.58it/s] 


✓ preferences.parquet salvato con 200000 coppie
